In [ ]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os

In [ ]:
dataset = fo.Dataset.from_images_dir('E:\\AI_Challenge\\keyframes-2', name="AIC_Dataset_Set2", tags=None, recursive=True)

In [ ]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    sample.save()

In [ ]:
for sample in dataset:
    object_path = f"E:\\AI_Challenge\\object-2\\{sample['video']}\\{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()

In [ ]:
all_keyframe = glob('E:\\AI_Challenge\\keyframes-2\\*\\*.jpg')
##Nên đổi lại thành <địa chỉ tới file keyframes\\*\\*.jpg
video_keyframe_dict = {}
all_video = glob('E:\\AI_Challenge\\keyframes-2\\*')
##tương tự, nên đổi thành <địa chỉ tới file keyframes\\*\\*
all_video = [v.rsplit('\\',1)[-1] for v in all_video]

In [ ]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

In [ ]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

In [ ]:
embedding_dict = {}
for v in all_video:
    clip_path = f'E:\\AI_Challenge\\clip-features-32-2\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]

In [ ]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)

In [ ]:
import numpy as np

# Chuyển đổi danh sách clip_embeddings thành mảng numpy
clip_embeddings = np.array(clip_embeddings)

# Sau đó chuyển đổi kiểu dữ liệu thành float32
clip_embeddings = clip_embeddings.astype(np.float32)


In [ ]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim_1",
)

In [2]:
fob.compute_visualization(
     dataset,
     embeddings=clip_embeddings,
     brain_key="img_viz"
)##Note từ Đức Trung: Cái này m phải tải numpy về trc, trong cmd của anacoda ấy, tải bản 1.26.4 hay thấp hơn 2.0 là đc, mặc định là 2.1 ko chạy đc đâu, tải xong từ sẽ phải chạy lại từ đầu :))

In [ ]:
import pandas as pd
from glob import glob
import os

# Lấy đường dẫn tới tất cả các tệp CSV trong folder
csv_folder = 'E:\\AI_Challenge\\mao-keyframes-2'
csv_files = glob(os.path.join(csv_folder, "*.csv"))

# Đọc và hợp nhất tất cả các tệp CSV vào một DataFrame
map_keyframe_data = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Hiển thị vài dòng đầu để kiểm tra
print(map_keyframe_data.head())

In [ ]:
# Tích hợp lại từ Bước 1 và Bước 2
def get_frame_idx(n_value, map_data):
    """
    Trả về frame_idx tương ứng với giá trị n từ map keyframe data.
    
    Args:
        n_value: Giá trị `n` từ file keyframes JPG.
        map_data: DataFrame chứa thông tin `n`, `frame_idx`.
    
    Returns:
        frame_idx tương ứng với n_value hoặc None nếu không tìm thấy.
    """
    result = map_data[map_data['n'] == n_value]
    if not result.empty:
        return result.iloc[0]['frame_idx']
    return None

In [ ]:
import os

def extract_n_from_filename(filepath):
    """
    Trích xuất giá trị n từ tên file keyframe (jpg).
    
    Args:
        filepath: Đường dẫn tới file JPG.
    
    Returns:
        n_value: Giá trị n được trích xuất từ tên file.
    """
    filename = os.path.basename(filepath)
    n_value = os.path.splitext(filename)[0].split('_')[-1]  # Giả sử n là phần cuối sau dấu _
    return int(n_value)


In [ ]:
import fiftyone as fo
import pandas as pd
from glob import glob
import os

# Đọc tất cả các tệp CSV và hợp nhất chúng
csv_folder = 'E:\\AI_Challenge\\mao-keyframes-2'
csv_files = glob(os.path.join(csv_folder, "*.csv"))
map_keyframe_data = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Lấy tất cả các tệp keyframe JPG
all_keyframes = glob('E:\\AI_Challenge\\keyframes-2\\*\\*.jpg')

# Tải dataset đã tồn tại
dataset = fo.load_dataset('AIC_Dataset_Set2')

# Duyệt qua từng sample trong dataset và ánh xạ frame_idx từ giá trị n
for sample in dataset:
    # Lấy giá trị n từ tên file JPG
    n_value = extract_n_from_filename(sample['filepath'])
    
    # Tra cứu frame_idx từ n
    frame_idx = get_frame_idx(n_value, map_keyframe_data)
    
    # Lưu lại thông tin frame_idx vào sample
    sample["frame_idx"] = frame_idx
    sample.save()

# Kiểm tra dataset sau khi cập nhật
for sample in dataset:
    print(f"Video: {sample['video']}, Frame ID: {sample['frameid']}, Frame IDX: {sample['frame_idx']}")


In [ ]:
session=fo.launch_app(dataset, auto=False)

In [ ]:
session.open_tab()